In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import sys
import random
from sklearn.preprocessing import StandardScaler
from IPython.display import display, Math
from torch.utils.data import DataLoader, Dataset

sys_epsilon = sys.float_info.epsilon

# Data Preprocessing

In this section, we preprocess the CFD data collected from DSM-based LES of flow past a sphere at $\mathcal{R}e=10^3$, $5\times 10^3$, and $10^4$. The data for each Reynolds number is collected from $t=300$ to $320$. From $t=300$ to $310$, data is collected at every ten variable time-steps. However, from $t=310$ to $320$, data is collected with $\Delta t =1$.

In [2]:
paths = ["/home/hmarefat/scratch/torchFOAM/Case_dS/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R53/postProcessing/fieldData.dat",
         "/home/hmarefat/scratch/torchFOAM/Case_dS_R4/postProcessing/fieldData.dat"]
nCols = 26
names = ['_R3', '_R53', '_R4']

In [3]:
def returnRandmIndx(maxIndx, count):
    return np.array(random.sample(range(1,maxIndx), count))

In [15]:
random.seed(35)

count = 1000000
Indx_R3 = returnRandmIndx(4956526, count)
Indx_R53 = returnRandmIndx(5900626, count)
Indx_R4 = returnRandmIndx(4012425, count)

In [5]:
Indx_R3[0:10]

array([4602784, 2813286, 1104994, 2856052, 1292255, 2398361, 3621576,
       2115176, 4754160,  518511])

In [6]:
random.shuffle(Indx_R3)

In [7]:
Indx_R3[0:10]

array([1433998, 3617811, 4647835, 4488912, 3146260, 4916233, 2427443,
       3525259, 1800030,  748959])

In [8]:
def splitterIndx(indx):
    seen = indx[:int(0.7*indx.shape[0])]
    unseen = indx[int(0.7*indx.shape[0]):]
    
    return seen, unseen

In [16]:
index_data = []

for indx in [Indx_R3, Indx_R53, Indx_R4]:
    seen, unseen = splitterIndx(indx)
    index_data.append((seen, unseen))

index_array = np.array(index_data, dtype=object)

In [17]:
index_array[2,0].shape, index_array[2,1].shape

((700000,), (300000,))

In [18]:
def scaler(name, ds):
    scaler = StandardScaler()
    scaler.fit(ds)
    np.savetxt(f'../processedDatasets/{name}_means.txt',scaler.mean_)
    np.savetxt(f'../processedDatasets/{name}_scales.txt',scaler.scale_)
    ds_norm = scaler.transform(ds)   
    np.savetxt(f'../processedDatasets/{name}_norm.txt', ds_norm, fmt='%.18e')
    np.savetxt(f'../processedDatasets/{name}.txt', ds, fmt='%.18e')

In [19]:
data = []

for i, path in enumerate(paths):
    name = path.split('/')[-1][:-4]+names[i]
    print(f'Starting to preprocess dataset {name}')
    data.clear()
    
    with open(path, "r") as f:
        next(f)
        for line in f:
            l = line.split()
            try:
                data.append([float(x) for x in l]) #(l)
            except ValueError as e:
                print(f"Error converting line to float: {line.strip()} - {e}")
                continue
                
    #data.pop(0)
    print('Reading raw file is done!')
    ds = np.array(data) 
    
    ds_seen = ds[index_array[i,0]]
    ds_unseen = ds[index_array[i,1]]
    
    scaler(name+'_seen', ds_seen)
    scaler(name+'_unseen', ds_unseen)
    
    print('\n')

Starting to preprocess dataset fieldData_R3
Reading raw file is done!


Starting to preprocess dataset fieldData_R53
Reading raw file is done!


Starting to preprocess dataset fieldData_R4
Reading raw file is done!




# Data Preparation

In [94]:
headers = ["t",                                             # time
           "X", "Y", "Z",                                   # spacial coordinates
           "Ux", "Uy", "Uz",                                # velocity components
           "G1", "G2", "G3", "G4", "G5", "G6",              # velocity gradient tensor components
           "S1", "S2", "S3", "S4", "S5", "S6",              # strain rate tensor compnents
           "UUp1", "UUp2", "UUp3", "UUp4", "UUp5", "UUp6",  # resolved Reynolds stress tensor components
           "Cs"]                                            # Smagorinsky coefficient

### Dataset Loading

#### $\mathcal Re = 10^3$

In [95]:
# Re = 10^3 seen
dSn_R103_seen_means = pd.read_csv('../processedDatasets/fieldData_R3_seen_means.txt', sep=' ', names=headers) 
dSn_R103_seen_scales = pd.read_csv('../processedDatasets/fieldData_R3_seen_scales.txt', sep=' ', names=headers) 
dSn_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen_norm.txt', sep=' ', names=headers)
dS_R103_seen = pd.read_csv('../processedDatasets/fieldData_R3_seen.txt', sep=' ', names=headers)

# Re = 10^3 unseen
dSn_R103_unseen_means = pd.read_csv('../processedDatasets/fieldData_R3_unseen_means.txt', sep=' ', names=headers) 
dSn_R103_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R3_unseen_scales.txt', sep=' ', names=headers) 
dSn_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen_norm.txt', sep=' ', names=headers)
dS_R103_unseen = pd.read_csv('../processedDatasets/fieldData_R3_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 5\times 10^3$

In [96]:
# Re = 5 x 10^3 seen
dSn_R503_seen_means = pd.read_csv('../processedDatasets/fieldData_R53_seen_means.txt', sep=' ', names=headers) 
dSn_R503_seen_scales = pd.read_csv('../processedDatasets/fieldData_R53_seen_scales.txt', sep=' ', names=headers) 
dSn_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen_norm.txt', sep=' ', names=headers)
dS_R503_seen = pd.read_csv('../processedDatasets/fieldData_R53_seen.txt', sep=' ', names=headers)

# Re = 5 x 10^3 unseen
dSn_R503_unseen_means = pd.read_csv('../processedDatasets/fieldData_R53_unseen_means.txt', sep=' ', names=headers) 
dSn_R503_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R53_unseen_scales.txt', sep=' ', names=headers) 
dSn_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen_norm.txt', sep=' ', names=headers)
dS_R503_unseen = pd.read_csv('../processedDatasets/fieldData_R53_unseen.txt', sep=' ', names=headers)

#### $\mathcal Re = 10^4$

In [97]:
# Re = 10^4 seen
dSn_R104_seen_means = pd.read_csv('../processedDatasets/fieldData_R4_seen_means.txt', sep=' ', names=headers) 
dSn_R104_seen_scales = pd.read_csv('../processedDatasets/fieldData_R4_seen_scales.txt', sep=' ', names=headers) 
dSn_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen_norm.txt', sep=' ', names=headers)
dS_R104_seen = pd.read_csv('../processedDatasets/fieldData_R4_seen.txt', sep=' ', names=headers)

# Re = 10^4 unseen
dSn_R104_unseen_means = pd.read_csv('../processedDatasets/fieldData_R4_unseen_means.txt', sep=' ', names=headers) 
dSn_R104_unseen_scales = pd.read_csv('../processedDatasets/fieldData_R4_unseen_scales.txt', sep=' ', names=headers) 
dSn_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen_norm.txt', sep=' ', names=headers)
dS_R104_unseen = pd.read_csv('../processedDatasets/fieldData_R4_unseen.txt', sep=' ', names=headers)

# Model Configuration Setup

In this section, we consider the different model configurations as presented in the following table.

$$
\begin{array}{|l|c|c|c|c|}
    \hline
    \textbf{Model} & \textbf{Inputs} & \textbf{No. of Inputs} & \textbf{Outputs} & \textbf{No. of Outputs} \\
    \hline
    \mathbf{M1} & u_i \, \text{and} \, \mathcal{S}_{ij} & 9 & c_s & 1\\
    \mathbf{M2} & \mathcal{G}_{ij} \, \text{and} \, \mathcal{S}_{ij} & 12 & c_s & 1 \\
    \mathbf{M3} & u_i \, \text{and} \, \tau^{'}_{ij} & 9 & c_s & 1 \\
    \mathbf{M4} & \mathcal{G}_{ij} \, \text{and} \, \tau^{'}_{ij} & 12 & c_s & 1 \\
    \hline
\end{array}
$$

In [98]:
R103 = dSn_R103_seen
R503 = dSn_R503_seen
R104 = dSn_R104_seen

R103_un = dSn_R103_unseen
R503_un = dSn_R503_unseen
R104_un = dSn_R104_unseen

In [99]:
M1_headers = ['Ux', 'Uy', 'Uz', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M2_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'S1',  'S2', 'S3', 'S4', 'S5', 'S6', 'Cs']
M3_headers = ['Ux', 'Uy', 'Uz', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']
M4_headers = ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'UUp1',  'UUp2', 'UUp3', 'UUp4', 'UUp5', 'UUp6', 'Cs']

M1_103 = R103.filter(M1_headers, axis=1)
M2_103 = R103.filter(M2_headers, axis=1)
M3_103 = R103.filter(M3_headers, axis=1)
M4_103 = R103.filter(M4_headers, axis=1)

M1_503 = R503.filter(M1_headers, axis=1)
M2_503 = R503.filter(M2_headers, axis=1)
M3_503 = R503.filter(M3_headers, axis=1)
M4_503 = R503.filter(M4_headers, axis=1)

M1_104 = R104.filter(M1_headers, axis=1)
M2_104 = R104.filter(M2_headers, axis=1)
M3_104 = R104.filter(M3_headers, axis=1)
M4_104 = R104.filter(M4_headers, axis=1)

M1_103_test = R103_un.filter(M1_headers, axis=1)
M2_103_test = R103_un.filter(M2_headers, axis=1)
M3_103_test = R103_un.filter(M3_headers, axis=1)
M4_103_test = R103_un.filter(M4_headers, axis=1)

M1_503_test = R503_un.filter(M1_headers, axis=1)
M2_503_test = R503_un.filter(M2_headers, axis=1)
M3_503_test = R503_un.filter(M3_headers, axis=1)
M4_503_test = R503_un.filter(M4_headers, axis=1)

M1_104_test = R104_un.filter(M1_headers, axis=1)
M2_104_test = R104_un.filter(M2_headers, axis=1)
M3_104_test = R104_un.filter(M3_headers, axis=1)
M4_104_test = R104_un.filter(M4_headers, axis=1)

# Model Training

In [100]:
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [101]:
dt = M1_103
dt_name = namestr(M1_103, globals())[0]

In [102]:
output_size = 1
input_size = dt.shape[1] - output_size 
neurons_per_layer = [60, 60, 60, 60, 60] 
hidden_layers = len(neurons_per_layer) 

split_sz = 0.8
mask = np.random.rand(len(dt)) < split_sz
train = dt[mask].reset_index(drop=True) 
val = dt[~mask].reset_index(drop=True)

In [103]:
train

,Ux,Uy,Uz,S1,S2,S3,S4,S5,S6,Cs
0,0.256586,-1.037380,0.032353,0.179013,-0.165670,-0.044446,0.681956,-0.266878,0.125857,5.377695
1,0.405770,0.198436,-0.115980,-0.009625,-0.022173,0.019609,0.059045,-0.064982,-0.034462,-0.518536
2,0.483732,0.135156,0.074298,0.036092,-0.062360,0.089373,-0.129665,0.344781,0.088810,0.014761
3,0.273207,-0.739299,1.223583,-0.124293,0.093676,-0.239736,-0.033031,-0.629884,0.294321,-0.474977
4,0.441165,0.000057,-0.644073,0.283349,0.025366,-0.111974,0.030496,0.616915,-0.321619,0.235588
...,...,...,...,...,...,...,...,...,...,...
559945,0.071412,-1.093466,-0.059101,0.386222,0.923274,-0.021180,-0.307615,-0.018244,-0.165543,-0.330632
559946,0.352909,-0.612266,0.615389,-0.082247,0.082736,-0.101877,0.006784,-0.386773,0.109398,-0.575114
559947,0.341136,0.675160,-0.454757,0.063029,0.142742,-0.087494,-0.122395,0.264377,0.056332,0.228152
559948,0.398007,-0.270934,1.094988,-0.153261,0.080088,-0.549146,-0.196775,-0.392620,0.566632,-0.424977


In [104]:
val

,Ux,Uy,Uz,S1,S2,S3,S4,S5,S6,Cs
0,0.151697,0.896921,-0.611916,-1.198093,0.063899,-1.716066,1.564544,0.378995,-1.276389,0.470168
1,0.368097,0.166111,-0.022698,-0.025856,0.045116,0.079135,0.163089,-0.054048,-0.116564,0.007707
2,0.836546,0.668290,-0.624432,-0.288837,-0.066660,0.099825,0.226816,0.494591,-0.286807,0.101043
3,0.379332,0.265818,-0.731130,-0.153895,0.041697,-0.012102,-0.120492,-0.337773,0.254866,-0.515654
4,0.399596,-0.191945,-0.057558,0.043972,0.020775,0.074682,0.064291,0.053046,-0.113168,0.092444
...,...,...,...,...,...,...,...,...,...,...
140045,0.408677,-0.081545,-0.603177,-0.229717,0.203787,-0.051251,0.228232,-0.077136,0.201801,0.161237
140046,0.387106,-0.819364,0.773560,0.190289,-0.108191,0.120848,-0.114782,0.510844,-0.121200,0.247018
140047,-0.347242,-0.102225,0.326274,-0.727576,1.153021,-0.110290,1.571345,0.020547,-0.708087,-0.220649
140048,0.353708,-0.069333,0.053761,-0.022119,-0.033151,-0.057264,-0.032834,-0.065366,0.068294,-0.208105


In [121]:
class MyDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __getitem__(self, index):
        # Ensure all indices are valid
        if index < len(self.data):
            return torch.tensor(self.data.iloc[index].values, dtype=torch.float64)
        else:
            raise IndexError(f"Index {index} out of range for dataset with length {len(self.data)}")

    def __len__(self):
        return len(self.data)

In [122]:
batch_sz_trn = 4096
batch_sz_val = int(batch_sz_trn / 4)

train_dataset = MyDataset(train)
val_dataset = MyDataset(val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_sz_trn, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=batch_sz_val, shuffle=True)

In [123]:
data_iter = iter(train_loader)
next(data_iter)[0]

tensor([ 0.1995,  0.4135, -1.9074, -0.2632,  0.6383, -0.7686,  1.4252, -0.9879,
        -0.5295, -0.4727], dtype=torch.float64)

In [124]:
def coeff_determination(y_true, y_pred):
    SS_res = torch.sum(torch.square( y_true - y_pred ))
    SS_tot = torch.sum(torch.square( y_true - torch.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + sys_epsilon) )

In [125]:
class MLPModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, neurons_per_layer):
        super(MLPModel, self).__init__()

        layers = []
        layers.append(nn.Linear(input_size, neurons_per_layer[0]))
        layers.append(nn.ReLU())

        for i in range(1, hidden_layers):
            layers.append(nn.Linear(neurons_per_layer[i - 1], neurons_per_layer[i]))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(neurons_per_layer[-1], output_size))

        self.block = nn.Sequential(*layers)

    def forward(self, x):
        x = self.block(x)
        return x

In [126]:
model = MLPModel(input_size=input_size, 
                 output_size=output_size, 
                 hidden_layers=hidden_layers, 
                 neurons_per_layer=neurons_per_layer)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=3, gamma=0.2)

device = torch.device("cuda")
model.to(device)
model.double()

MLPModel(
  (block): Sequential(
    (0): Linear(in_features=9, out_features=60, bias=True)
    (1): ReLU()
    (2): Linear(in_features=60, out_features=60, bias=True)
    (3): ReLU()
    (4): Linear(in_features=60, out_features=60, bias=True)
    (5): ReLU()
    (6): Linear(in_features=60, out_features=60, bias=True)
    (7): ReLU()
    (8): Linear(in_features=60, out_features=60, bias=True)
    (9): ReLU()
    (10): Linear(in_features=60, out_features=1, bias=True)
  )
)

In [127]:
class EarlyStopper:
    def __init__(self, patience=1, path=None):
        self.patience = patience
        self.path = path
        self.counter = 0
        self.min_val_loss = float('inf')

    def early_stop(self, model_stat, val_loss):
        if val_loss < self.min_val_loss:
            torch.save(model_stat, self.path)
            self.min_val_loss = val_loss
            self.counter = 0
        elif val_loss > (self.min_val_loss + sys_epsilon):
            self.counter += 1
            if self.counter >= self.patience:
                print('+++ Early Stopping is reached! +++')
                return True
        return False

In [128]:
def log_epoch_info(epoch, epochs, Loss_train, coeff_train, Loss_val, coeff_val):
    message = (
        f"Epoch: {epoch} / {epochs}, \n"
        f"Train -- Loss: {Loss_train}, Coeff: {coeff_train} \n"
        f"Val   -- Loss: {Loss_val}, Coeff: {coeff_val} \n\n"
    )
    print(message)

In [ ]:
epochs = 6000
best = 1e6
PATH = f"./best_model_{dt_name}.pt"

early_stopper = EarlyStopper(patience=50, path=PATH)
                             
for epoch in range(epochs):
    model.train()
    Loss_train = 0 
    Loss_val = 0
    coeff_train = 0
    coeff_val = 0
    with tqdm(train_loader, unit="batch") as trainer:
        for batch in trainer:
            trainer.set_description("Train")
            train_feat = batch[:, 0:-1].to(device)
            train_labs = batch[:, -1].to(device)
            train_pred = model(train_feat).squeeze()
            train_loss = torch.nn.functional.mse_loss(train_pred, train_labs)
            train_coef = coeff_determination(train_labs, train_labs) 

            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            Loss_train += train_loss.item()
            coeff_train += train_coef

        Loss_train /= batch_sz_trn
        coeff_train /= batch_sz_trn

    model.eval()
    with tqdm(val_loader, unit="batch") as validator:
        for batch in validator:
            validator.set_description("Valdt")
            val_feat = batch[:, 0:-1].to(device)
            val_labs = batch[:, -1].to(device)
            val_pred = model(val_feat).squeeze()
            val_loss = torch.nn.functional.mse_loss(val_pred, val_labs)
            val_coef = coeff_determination(val_labs, val_labs) 

            Loss_val += val_loss.item()
            coeff_val += val_coef

        Loss_val /= batch_sz_val
        coeff_val /= batch_sz_val
    
    if early_stopper.early_stop(model.state_dict(), Loss_val):             
        break
    
    log_epoch_info(epoch, epochs, Loss_train, coeff_train, Loss_val, coeff_val)

Valdt: 100%|██████████| 137/137 [00:05<00:00, 26.51batch/s]


Epoch: 0 / 6000, 
Train -- Loss: 0.02809513001312743, Coeff: 0.033447265625 
Val   -- Loss: 0.08414723576210915, Coeff: 0.1337890625 




Valdt: 100%|██████████| 137/137 [00:05<00:00, 26.24batch/s]


Epoch: 1 / 6000, 
Train -- Loss: 0.018031762073992184, Coeff: 0.033447265625 
Val   -- Loss: 0.06374639027249876, Coeff: 0.1337890625 




Valdt: 100%|██████████| 137/137 [00:05<00:00, 26.64batch/s]


...

Valdt: 100%|██████████| 137/137 [00:05<00:00, 26.36batch/s]


Epoch: 462 / 6000, 
Train -- Loss: 0.0016394942711171706, Coeff: 0.033447265625 
Val   -- Loss: 0.008426160574866923, Coeff: 0.1337890625 




Valdt: 100%|██████████| 137/137 [00:05<00:00, 26.60batch/s]


Epoch: 463 / 6000, 
Train -- Loss: 0.00167413557479955, Coeff: 0.033447265625 
Val   -- Loss: 0.008659634762484384, Coeff: 0.1337890625 




Train:  94%|█████████▍| 129/137 [00:18<00:01,  6.89batch/s]